<a href="https://colab.research.google.com/github/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/blob/main/Transfer_learning_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading and becoming one with the data

10% of 10 food classes from Food101

In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref=zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2025-01-24 01:04:20--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 74.125.137.207, 142.250.101.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   146MB/s    in 1.1s    

2025-01-24 01:04:21 (146 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [ ]:
from posixpath import dirname
# lets walk through the directories and find the files

import os

for dirpath,dirname,filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirname)} diectories and {len(filenames)} images in {dirpath}")

There are 2 diectories and 0 images in 10_food_classes_10_percent
There are 10 diectories and 0 images in 10_food_classes_10_percent/test
There are 0 diectories and 250 images in 10_food_classes_10_percent/test/hamburger
There are 0 diectories and 250 images in 10_food_classes_10_percent/test/ice_cream
There are 0 diectories and 250 images in 10_food_classes_10_percent/test/ramen
There are 0 diectories and 250 images in 10_food_classes_10_percent/test/steak
There are 0 diectories and 250 images in 10_food_classes_10_percent/test/chicken_curry
There are 0 diectories and 250 images in 10_food_classes_10_percent/test/pizza
There are 0 diectories and 250 images in 10_food_classes_10_percent/test/grilled_salmon
There are 0 diectories and 250 images in 10_food_classes_10_percent/test/fried_rice
There are 0 diectories and 250 images in 10_food_classes_10_percent/test/sushi
There are 0 diectories and 250 images in 10_food_classes_10_percent/test/chicken_wings
There are 10 diectories and 0 imag

## Creating Data Loaders
we will use ImageDataGenerator from keras to load our image data into batches

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SHAPE=(224,224)
BATCH_SIZE=32

train_dir="10_food_classes_10_percent/train/"
test_dir="10_food_classes_10_percent/test"

train_datagen=ImageDataGenerator(rescale=1/255.)
test_datagen=ImageDataGenerator(rescale=1/255.)

print("Training Images")
train_data_10_percent=train_datagen.flow_from_directory(train_dir,
                                                        target_size=IMG_SHAPE,
                                                        batch_size=BATCH_SIZE,
                                                        class_mode="categorical")

print("Testing Images")
test_data_10_percent=test_datagen.flow_from_directory(test_dir,
                                                      target_size=IMG_SHAPE,
                                                      batch_size=BATCH_SIZE
                                                      ,class_mode="categorical")

Training Images
Found 750 images belonging to 10 classes.
Testing Images
Found 2500 images belonging to 10 classes.


In [ ]:
train_data_10_percent.class_indices

{'chicken_curry': 0,
 'chicken_wings': 1,
 'fried_rice': 2,
 'grilled_salmon': 3,
 'hamburger': 4,
 'ice_cream': 5,
 'pizza': 6,
 'ramen': 7,
 'steak': 8,
 'sushi': 9}

### Tracking experiment with tensorBoard

In [ ]:
# Create a tensorBoard callback

import datetime
def create_TB_callback(dirname,exp_name):
  log_dir=os.path.join(dirname,exp_name,datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tb_callback=tf.keras.callbacks.tensorBord(log_dir=log_dir)
  print(f"Saving tensorbord logs at {log_dir}")
  return tb_callback

In [ ]:
resnet_url="https://kaggle.com/models/google/resnet-v2/TensorFlow2/50-feature-vector/1"
efficient_net_url="https://www.kaggle.com/models/tensorflow/efficientnet/TensorFlow2/b0-feature-vector/1"

In [ ]:
# resnet model
import tensorflow_hub as hub
m = tf.keras.Sequential([
    hub.KerasLayer("https://kaggle.com/models/google/resnet-v2/TensorFlow2/50-feature-vector/1",
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
m.build([None, 224, 224, 3])  # Batch input shape.


ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7fd061225a50> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [ ]:
# creating model from TensorflowHUB

# import tensorflow_hub as hub
# m = tf.keras.Sequential([
#     hub.KerasLayer("https://www.kaggle.com/models/tensorflow/efficientnet/TensorFlow2/b0-feature-vector/1", #EfficientNetB0 feature vector (version 1)
#                    trainable=False),
#     tf.keras.layers.Dense(num_classes, activation='softmax')
# ])
# m.build([None, IMG_SHAPE, IMG_SHAPE, 3])  # Batch input shape.


In [ ]:
import tensorflow as tf
import tf_keras
import tensorflow_hub as hub
from tensorflow.keras import layers

def create_model(model_feature_url,num_classes=10):
  """returns uncompiled keras sequential model with model_url as feature extractor layer and dense output layer with num_classes output_neurons"""
  #download the pretrained model and save it as keras layer
  feature_extractor_layer=hub.KerasLayer(model_feature_url,
                                         trainable=False, # this freeze the already learned patterns
                                         name="feature_extractor_layer",
                                         input_shape=IMG_SHAPE+(3,))
  # create our model
  model=tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes,activation="softmax",name="output_layer")
  ])
  return model

## Creating and testing ResNet tensorflow hub feature extraction model

In [ ]:
train_data_10_percent.num_classes

10

In [ ]:
resnet_url="https://kaggle.com/models/google/resnet-v2/TensorFlow2/50-feature-vector/1"

resnet_model=create_model(resnet_url,num_classes=train_data_10_percent.num_classes)

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7fd086781050> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Define the image shape
IMG_SHAPE = (224, 224)

def create_model(model_feature_url, num_classes=10):
    """Returns an uncompiled Keras Sequential model with a feature extractor layer."""
    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(
        model_feature_url,
        trainable=False,  # Freeze the pretrained weights
        name="feature_extractor_layer",
        input_shape=IMG_SHAPE + (3,)
    )
    # Create the model
    model = tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dense(num_classes, activation="softmax", name="output_layer")
    ])
    return model

# Use a valid ResNet URL from TensorFlow Hub
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

# Assuming train_data_10_percent has a `num_classes` attribute
train_data_10_percent = {"num_classes": 10}  # Replace this with your dataset object

# Create the model
resnet_model = create_model(resnet_url, num_classes=train_data_10_percent["num_classes"])

# Print the model summary
resnet_model.summary()


ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7fd1042be2d0> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)